# Машинное обучение, Лабораторная работа №2

Преподаватель: Свинцов М. В.

Выполнили: Румянцев А. А. 368731, Чебаненко Д. А. 367609, R3341, МашОб 31 1.

**Цель работы**: построить модели линейной классификации и регрессии.
Инструменты: модели классификации (SGDClassifier) и регрессии (SGDRegressor) из пакета scikit-learn.
Содержание:
Для предложенных датасетов (один для классификации, другой для регрессии) построить соответствующие линейные модели. Для этого:
Воспользоваться результатами предыдущей лабораторной работы по препроцессингу

1.   Воспользоваться результатами предыдущей лабораторной работы по препроцессингу данных
2.   Разбить данные на обучающую и тестовую выборки (например, в соотношении 80/20).
3.   Построить классификатор и регрессор с различными вариантами архитектур.

4.   Обучить модели на обучающих частях выборки.

5.   Провести сравнительную оценку качества моделей на основе тестовой выборки.   


Архитектура
Замечание про архитектуры (классификатор):

1. В качестве loss-функции рассмотреть: perceptron, hinge и squared_hinge. Не использовать log_loss! Это будет в следующей лабораторной!
2. Рассмотреть различные варианты регуляризации (penalty): L1, L2 или Elastic Net.
3. Используя GridSearchCV, определить лучший классификатор.
4. (бонус) можно попробовать minibatch optimization при помощи функции partial_fit.


Замечание про архитектуры (регрессор):

1. В качестве loss-функции рассмотреть: squared_error, huber и epsilon_insensitive.
2. Рассмотреть различные варианты регуляризации (penalty): L1, L2 или Elastic Net.
3. Используя GridSearchCV, определить лучший регрессор.
4. (бонус) можно попробовать minibatch optimization при помощи функции partial_fit.


# Подготовка



# Импорт необходимых библиотек

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, make_scorer, mean_absolute_error
from sklearn.preprocessing import StandardScaler


# Подключение диска

Считаем датасет созданный в лабораторной работе №1. Поэтому данные не нужно обрабатывать.  

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
classification_df = pd.read_csv('drive/My Drive/new_heart.csv')
classification_df

,oldpeak,age_heart_rate_reserve,physical_fitness,sex,cp,exang,slope,ca,thal,target
0,1.087338,-0.606321,0.298233,1,3,0,0,0,1,1
1,2.122573,2.301531,-1.006925,1,2,0,0,0,2,1
2,0.310912,1.566541,-0.715457,0,1,0,2,0,2,1
3,-0.206705,0.374341,-1.108028,1,1,0,2,0,2,1
4,-0.379244,-0.002645,-0.809770,0,0,1,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...
298,-0.724323,-0.833661,1.122947,0,0,1,1,0,3,0
299,0.138373,0.066187,-0.342777,1,3,0,1,0,3,0
300,2.036303,-0.990186,0.560774,1,0,0,1,2,3,0
301,0.138373,-1.022528,1.085560,1,0,1,1,1,3,0


##Разделение на обучающую и тестовую выборки 80 на 20

In [ ]:
x_class = classification_df.drop("target", axis=1)
y_class = classification_df["target"]

x_class_train, x_class_test, y_class_train, y_class_test = train_test_split(x_class, y_class, test_size=0.2, random_state=100)

##Гиперпараметры и обучение

## `loss` - функция потерь используемая для оптимизации модели.

1. `perceptron` - нет степени ошибки, штраф всегда одинаков

2. `hinge` - появляется уверенность, даже если предсказание верное, но все равно на границе, все равно штрафуем модель

3. `squared_hinge` - тоже самое, что и `hinge`, но возведением в квадтрат сильнее штрафует серьезные ошибки

## `penalty` - это метод предотврещения переобучения модели

1. `L1` - добавляет сумму значений коэффициентов по модулю
2. `L2` - добавляет сумму квадратов коэффициентов
3. `elasticnet` - сочетанфие `L1` и `L2`

In [ ]:
classifier = SGDClassifier()
classifier_params = {
    "loss": ["perceptron", "hinge", "squared_hinge"],
    "penalty": ["l1", "l2", "elasticnet"],
    "alpha": [0.0001, 0.001, 0.01, 0.1],
    "max_iter": [10000],
    "random_state": [42]
}
sgd_grid = GridSearchCV(classifier, classifier_params, cv=4, scoring='accuracy', n_jobs=-1)

sgd_grid.fit(x_class_train, y_class_train)




GridSearchCV(cv=4, estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'loss': ['perceptron', 'hinge', 'squared_hinge'],
                         'max_iter': [10000],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'random_state': [42]},
             scoring='accuracy')

##Лучшая модель

In [ ]:
print(f"Best {type(classifier).__name__} parameters: {sgd_grid.best_params_}\n" +\
      f"Accuracy on all folds: {sgd_grid.best_score_}\n" +\
      f"Accuracy on test set: " +\
      f"{accuracy_score(y_class_test, sgd_grid.best_estimator_.predict(x_class_test))}")

Best SGDClassifier parameters: {'alpha': 0.01, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'l2', 'random_state': 42}
Accuracy on all folds: 0.8388661202185793
Accuracy on test set: 0.8360655737704918


##Все модели

In [ ]:
RFresults = sgd_grid.cv_results_

results_df = pd.DataFrame(results)
results = pd.DataFrame(sgd_grid.cv_results_)

results_summary = results[[
    'param_loss', 'param_penalty', 'param_alpha',
    'mean_test_score'
]]

results_summary = results_summary.rename(columns={
    'param_loss': 'loss',
    'param_penalty': 'penalty',
    'param_alpha': 'alpha',
    'mean_test_score': 'mean_accuracy',
})


results_summary = results_summary.sort_values('mean_accuracy', ascending=False)

results_with_predictions = results_summary.copy()


test_accuracies = []

for model_idx in range(len(sgd_grid.cv_results_['params'])):

    model = sgd_grid.estimator.set_params(**sgd_grid.cv_results_['params'][model_idx])

    model.fit(x_class_train, y_class_train)

    y_pred = model.predict(x_class_test)

    accuracy = accuracy_score(y_class_test, y_pred)
    test_accuracies.append(accuracy)

results_with_predictions['test_accuracy'] = test_accuracies

print(results_with_predictions)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic

             loss     penalty   alpha  mean_accuracy  test_accuracy
22          hinge          l2  0.0100       0.838866       0.885246
23          hinge  elasticnet  0.0100       0.838866       0.721311
31          hinge          l2  0.1000       0.838798       0.721311
21          hinge          l1  0.0100       0.826366       0.836066
32          hinge  elasticnet  0.1000       0.818101       0.737705
5           hinge  elasticnet  0.0001       0.805396       0.901639
17  squared_hinge  elasticnet  0.0010       0.797541       0.754098
13          hinge          l2  0.0010       0.793443       0.639344
18     perceptron          l1  0.0100       0.793238       0.737705
25  squared_hinge          l2  0.0100       0.785041       0.819672
26  squared_hinge  elasticnet  0.0100       0.785041       0.885246
3           hinge          l1  0.0001       0.784904       0.786885
24  squared_hinge          l1  0.0100       0.776708       0.852459
4           hinge          l2  0.0001       0.77

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


## Выводы по классификации

Как мы можем заметить оптимальным сочетанием гиперпараметров для нашей модели является 'alpha': 0.01, 'loss': 'hinge', 'max_iter': 10000, 'penalty': 'elasticnet'

параметр `alpha` - отвечает за силу регуляризацию слишком большое значение, такое как 0.1 будет слишком сильно наказывать модель за большые веса и не давать ей полноценно обучаться, слишком же маленькие значения, наоборот гасят пенальти делая его малозначимым.

Лучшем penalty оказался `elasticnet` - который является сочетанием `L1` и `L2`


In [ ]:
regression_df = pd.read_csv("drive/My Drive/clean_advertising.csv")

x_reg = regression_df.drop("sales", axis=1)
y_reg = regression_df["sales"]

x_reg_train, x_reg_test, y_reg_train, y_reg_test = train_test_split(x_reg, y_reg, test_size=0.2, random_state=100)

squared_error - обычная ошибка

huber - комбинация квадратичной ошибки и абсолютной ошибки

epsilon_insensitive - ошибки меньше порога не учитываются


In [ ]:


reg_params = {
    "loss": ["squared_error", "huber", "epsilon_insensitive"],
    "penalty": ["l1", "l2", "elasticnet"],
    "alpha": [0.0001, 0.001, 0.01],
    "max_iter": [1000],
    "random_state": [42]
}

reger = SGDRegressor()

reg_grid = GridSearchCV(reger, reg_params, cv = 4, scoring = 'neg_mean_squared_error', n_jobs=-1)
reg_grid.fit(x_reg_train, y_reg_train)

best_reg = reg_grid.best_estimator_
y_reg_pred = best_reg.predict(x_reg_test)
reg_mse = mean_squared_error(y_reg_test, y_reg_pred)
reg_r2 = r2_score(y_reg_test, y_reg_pred)

print("Лучший регрессор:", best_reg)
print("Выбранная функция потерь:", best_reg.loss)
print("Среднеквадратичная ошибка на тестовой выборке:", reg_mse)
print("Коэффициент детерминации (R2):", reg_r2)


Лучший регрессор: SGDRegressor(alpha=0.01, random_state=42)
Выбранная функция потерь: squared_error
Среднеквадратичная ошибка на тестовой выборке: 5.123449747470855
Коэффициент детерминации (R2): 0.8079930472070475


In [ ]:
results = pd.DataFrame(reg_grid.cv_results_)

results_summary = results[[
    'param_loss', 'param_penalty', 'param_alpha',
    'mean_test_score'
]]

results_summary = results_summary.rename(columns={
    'param_loss': 'loss',
    'param_penalty': 'penalty',
    'param_alpha': 'alpha',
    'mean_test_score': 'mean_mse',
})



results_with_predictions = results_summary.copy()


test_mse = []
test_r2 = []

for model_idx in range(len(reg_grid.cv_results_['params'])):

    model = reg_grid.estimator.set_params(**reg_grid.cv_results_['params'][model_idx])

    model.fit(x_reg_train, y_reg_train)

    y_pred = model.predict(x_reg_test)

    mse = mean_squared_error(y_reg_test, y_pred)

    r2 = r2_score(y_reg_test, y_pred)

    test_mse.append(mse)
    test_r2.append(r2)

results_with_predictions['test_mse'] = test_mse
results_with_predictions['test_r2'] = test_r2
results_with_predictions = results_with_predictions.sort_values('test_r2', ascending=False)
print(results_with_predictions)

                   loss     penalty   alpha    mean_mse    test_mse   test_r2
19        squared_error          l2  0.0100   -2.750403    5.123450  0.807993
20        squared_error  elasticnet  0.0100   -2.756597    5.137817  0.807455
18        squared_error          l1  0.0100   -2.796503    5.202591  0.805027
10        squared_error          l2  0.0010   -2.796500    5.233578  0.803866
11        squared_error  elasticnet  0.0010   -2.797320    5.235276  0.803802
9         squared_error          l1  0.0010   -2.802016    5.244970  0.803439
1         squared_error          l2  0.0001   -2.803049    5.248167  0.803319
2         squared_error  elasticnet  0.0001   -2.803133    5.248340  0.803313
0         squared_error          l1  0.0001   -2.803609    5.249320  0.803276
26  epsilon_insensitive  elasticnet  0.0100   -3.184927    5.265202  0.802681
25  epsilon_insensitive          l2  0.0100   -3.194327    5.267153  0.802608
24  epsilon_insensitive          l1  0.0100   -3.136241    5.271

У некоторых моделей видно, явное переобучение (очень плохие результаты на тестовой выборке). Лучшей же моделью оказалась модель с параметрами  {squared_error, l2, 0.0100}. Данные сортировались по коэффициенту детерминации, который показывает, насколько хорошо модель описывает данные.